In [ ]:
%pip install pandas scipy numpy scikit-learn matplotlib implicit tqdm requests py-spy

In [1]:
import pandas as pd
import numpy as np
import implicit
import requests
import gc
from tqdm import tqdm
from scipy.sparse import csr_matrix
from collections import defaultdict
from sklearn.model_selection import train_test_split

/home/jaz/miniconda3/envs/suggestions/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
actors_df = pd.read_csv("actors.csv", names=["actor"])
actor_to_id = {actor: idx for idx, actor in enumerate(actors_df["actor"])}
id_to_actor = {idx: actor for actor, idx in actor_to_id.items()}
safe_get_id = lambda x: actor_to_id.get(x, None)

# Load follows data, converting actor and target identifiers using the safe mapping
df = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                "follows.csv",
                usecols=[0, 1],
                names=["actor", "target"],
                converters={"actor": safe_get_id, "target": safe_get_id},
                chunksize=1_000_000,
            ),
            desc="Loading data",
            total=155,
            unit="chunks",
        )
    ]
)

print(f"Loaded {len(df)} follows")

# Drop rows where either actor_id or target_id is None (meaning they were not found in the map)
df = df.dropna().reset_index(drop=True)
df["actor"] = df["actor"].astype(int)
df["target"] = df["target"].astype(int)

print("Dropped rows with missing actors")

# Drop any actors that follow more than 5,000 other actors and remove them from both sides of the follows
actor_counts = df["actor"].value_counts()
df = df[df["actor"].isin(actor_counts[actor_counts <= 5_000].index)]
df = df[df["target"].isin(actor_counts[actor_counts <= 5_000].index)]
df = df.reset_index(drop=True)

print("Dropped actors following more than 5,000 other actors")

# Drop any actors that follow fewer than 5 other actors and remove them from both sides of the follows
actor_counts = df["actor"].value_counts()
df = df[df["actor"].isin(actor_counts[actor_counts >= 5].index)]
df = df[df["target"].isin(actor_counts[actor_counts >= 5].index)]
df = df.reset_index(drop=True)

print("Dropped actors following fewer than 5 other actors")

# Drop any actors with fewer than 5 followers and remove them from both sides of the follows
actor_counts = df["target"].value_counts()
df = df[df["actor"].isin(actor_counts[actor_counts >= 5].index)]
df = df[df["target"].isin(actor_counts[actor_counts >= 5].index)]
df = df.reset_index(drop=True)

print("Dropped actors with fewer than 5 followers")

df.head()

Loading data: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 155/155 [03:12<00:00,  1.24s/chunks]


Loaded 154396387 follows
Dropped rows with missing actors
Dropped actors following more than 5,000 other actors
Dropped actors following fewer than 5 other actors
Dropped actors with fewer than 5 followers


,actor,target
0,3587154,121789
1,5387692,3980839
2,4037694,3578576
3,3172038,612835
4,3127203,1249260


In [3]:
num_actors = len(actor_to_id)
num_targets = num_actors

train_df, test_df = train_test_split(df, test_size=0.2, random_state=11235)

# Create CSR matrices for both training and testing sets
def create_csr_matrix(data, num_actors):
    return csr_matrix(
        (np.ones(len(data)), (data["actor"], data["target"])), shape=(num_actors, num_actors)
    )

train_matrix = create_csr_matrix(train_df, len(actor_to_id))
test_matrix = create_csr_matrix(test_df, len(actor_to_id))

In [4]:
# Train and evaluate the model
def train_and_evaluate(als_params):
    model = implicit.als.AlternatingLeastSquares(**als_params)
    model.fit(train_matrix)

    # Evaluate the model
    def evaluate_model(model, test_matrix):
        precisions = []
        recalls = []
        # Sample 10,000 actors from the test
        sample_actors = np.random.choice(test_matrix.shape[0], 10_000, replace=False)
        
        for user_id in tqdm(sample_actors, desc="Evaluating model"):
            # Get the actors the user follows and check if the model thinks they are similar
            user_follows = test_matrix[user_id].indices
            if len(user_follows) == 0:
                continue

            # Truncate to the first 5 actors the user follows
            user_follows = user_follows[:5]

            for target_id in user_follows:
                # Get the top 10 most similar actors
                similar, _ = model.similar_items(target_id, N=10, filter_items=[target_id])

                # Calculate precision and recall
                hits = len(set(similar) & set(user_follows))
                precisions.append(hits / len(similar))
                recalls.append(hits / len(user_follows))

        return np.mean(precisions), np.mean(recalls)
    return model, evaluate_model(model, test_matrix)

In [5]:
# Grid search for the best hyperparameters
best_precision = 0
best_recall = 0
best_params = None

performance_list = []

for regularization in [0.01, 0.1]:
    for iterations in [25, 50, 100]:
        for factors in [128, 256]:
            als_params = {
                "factors": factors,
                "regularization": regularization,
                "iterations": iterations,
            }
            model, (precision, recall) = train_and_evaluate(als_params)
            
            precision_improvement_pct = 0
            recall_improvement_pct = 0
            if best_params is not None:
                precision_improvement_pct = (precision - best_precision) / best_precision * 100
                recall_improvement_pct = (recall - best_recall) / best_recall * 100
            if precision > best_precision:
                best_precision = precision
                best_recall = recall
                best_params = als_params
            del model
            gc.collect()

            performance_list.append({
                "factors": factors,
                "regularization": regularization,
                "iterations": iterations,
                "precision": precision,
                "recall": recall,
                "precision_improvement_pct": precision_improvement_pct,
                "recall_improvement_pct": recall_improvement_pct
            })
            
            print(f"Factors: {factors}, Regularization: {regularization}, Iterations: {iterations}, Precision: {precision :.5f} ({precision_improvement_pct :.2f}%), Recall: {recall: .5f} ({recall_improvement_pct :.2f}%)")


performances = pd.DataFrame(performance_list)

print("Performances:")
print(performances.sort_values("precision", ascending=False))

print(f"Best precision: {best_precision}, Best recall: {best_recall}, Best params: {best_params}")

# Train the model with the best hyperparameters
best_model, _ = train_and_evaluate(best_params)

Evaluating model: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:14<00:00, 681.36it/s]


Factors: 32, Regularization: 0.01, Iterations: 25, Precision: 0.00290 (0.00%), Recall:  0.00637 (0.00%)


Evaluating model: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:25<00:00, 399.85it/s]


Factors: 64, Regularization: 0.01, Iterations: 25, Precision: 0.00502 (73.00%), Recall:  0.01105 (73.34%)


Evaluating model: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:45<00:00, 219.83it/s]


...

Evaluating model: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:46<00:00, 213.66it/s]


Factors: 128, Regularization: 0.5, Iterations: 100, Precision: 0.00547 (-17.22%), Recall:  0.01200 (-16.13%)


Evaluating model: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:29<00:00, 111.33it/s]


Factors: 256, Regularization: 0.5, Iterations: 100, Precision: 0.00641 (-2.89%), Recall:  0.01407 (-1.68%)
Performances:
    factors  regularization  iterations  precision    recall  \
23      256            0.10         100   0.006604  0.014313   
27      256            0.50          25   0.006447  0.014188   
35      256            0.50         100   0.006413  0.014073   
31      256            0.50          50   0.006311  0.014074   
15      256            0.10          25   0.006235  0.013847   
11      256            0.01         100   0.006181  0.013165   
3       256            0.01          25   0.006063  0.013220   
19      256            0.10          50   0.006046  0.012896   
7       256            0.01          50   0.005997  0.012939   
10      128            0.01         100   0.005855  0.013012   
26      128            0.50          25   0.005727  0.012533   
18      128            0.10          50   0.005680  0.012616   
34      128            0.50         100   0.005

Evaluating model: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:25<00:00, 117.41it/s]


In [6]:
def get_actor_handle(actor_did):
    resp = requests.get(f"https://plc.jazco.io/{actor_did}")
    if "handle" in resp.json():
        return resp.json()["handle"]
    return actor_did


def get_actor_handles(actor_dids):
    resp = requests.post("https://plc.jazco.io/batch/by_did", json=actor_dids)
    for actor in resp.json():
        if "handle" in actor:
            yield actor["handle"]
        else:
            yield actor["did"]


def get_actor_dids(actor_handles):
    resp = requests.post("https://plc.jazco.io/batch/by_handle", json=actor_handles)
    for actor in resp.json():
        if "did" in actor:
            yield actor["did"]
        else:
            yield actor["handle"]


def get_actor_did(handle):
    resp = requests.get(f"https://plc.jazco.io/{handle}")
    return resp.json()["did"]


def aggregate_recommendations(interested_actors, N=20):
    """
    Aggregate recommendations for a new user based on their interested actors.

    :param interested_actors: List of actor IDs the new user might be interested in.
    :param N: Number of recommendations to make.
    :return: List of recommended actor handles and their scores.
    """
    actor_scores = defaultdict(float)
    actor_ids = [actor_to_id[actor] for actor in interested_actors]

    for actor_name in interested_actors:
        if actor_name in actor_to_id:
            actor_id = actor_to_id[actor_name]
            # Retrieve N most similar actors for each actor of interest
            similar_actors = best_model.similar_items(
                actor_id,
                N=N,
            )

            for similar_actor_id, score in zip(*similar_actors):
                if similar_actor_id not in actor_ids:  # Exclude the actor itself
                    if similar_actor_id in actor_scores:
                        # Boost the score of actors that are similar to multiple actors of interest
                        actor_scores[similar_actor_id] += score / len(interested_actors)
                    else:
                        actor_scores[similar_actor_id] += score

    # Normalize scores for each actor by the number of actors of interest
    for actor_id in actor_scores:
        actor_scores[actor_id] /= len(interested_actors)

    # Deduplicate actors of interest
    interested_actors = set(interested_actors)

    # Sort actors by aggregated score and select top N
    recommended_actors = sorted(actor_scores.items(), key=lambda x: x[1], reverse=True)[
        :N
    ]

    handles = list(
        get_actor_handles([id_to_actor[actor_id] for actor_id, _ in recommended_actors])
    )

    # Convert actor ids back to handles
    recommended_actors = [
        (handles[i], score) for i, (actor_id, score) in enumerate(recommended_actors)
    ]

    return recommended_actors


# A new user interested in specific actors
interested_actor_handles = [
    "shreyanjain.net",
    "mary.my.id",
]

# Convert actor handles to actor IDs
interested_actors = list(get_actor_dids(interested_actor_handles))

recommended_actors = aggregate_recommendations(interested_actors, N=20)

# Convert to a DataFrame for better visualization
recommended_actors_df = pd.DataFrame(recommended_actors, columns=["actor", "score"])
recommended_actors_df

,actor,score
0,futur.blue,0.663713
1,ovna.dev,0.631468
2,mackuba.eu,0.628405
3,samuel.bsky.team,0.616796
4,matthieu.bsky.team,0.446655
5,foysal.codes,0.439791
6,edavis.dev,0.439544
7,kyledrake.com,0.433332
8,haileyok.com,0.427974
9,jessica.bsky.team,0.425776


In [7]:
best_model.save("follow_recommendation_als.npz")

/home/jaz/miniconda3/envs/suggestions/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 48 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
